In [1]:
'''Suffix Array Construction Problem'''
'''In 1993, Udi Manber and Gene Myers introduced suffix arrays as a memory-efficient alternative to suffix trees.
To construct SuffixArray(Text), we first sort all suffixes of Text lexicographically, assuming that "$" comes first in the alphabet.
The suffix array is the list of starting positions of these sorted suffixes. For example,
#SuffixArray("panamabananas$") = (13, 5, 3, 1, 7, 9, 11, 6, 4, 2, 8, 10, 0, 12).
'''

'''
def SuffixArray(seq):
    suf = sorted([seq[i:] for i in xrange(len(seq))])
    posi=[]
    for n in range(len(suf)):
        posi+=[i for i in xrange(len(seq)) if seq[i:]==suf[n]]
    return posi
'''
#### rosalind solution ####
def SuffixArray(text): 
    '''
    Construct the suffix array of a string.
    INPUT:
    seq=AACGATAGCGGTAGA$
    OUTPUT=[15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5] # suffixes in alphabetical order identified by the position on their first letter in string.
    '''
    d = {}
    for i in xrange(len(text)):
        d[i] = text[i:]
    return sorted(d.keys(), key=lambda x: d[x])

seq='AACGATAGCGGTAGA$'
seq='PANAMABANANAS$'

f=open('../../Downloads/dataset_310_2.txt','r')
#f=open('../../Downloads/rosalind_ba9g.txt','r')
lines=f.read().splitlines()
f.close()
seq=lines[0]

print ', '.join(map(str,SuffixArray(seq)))

875, 184, 185, 370, 132, 488, 127, 198, 186, 371, 133, 456, 667, 660, 74, 351, 870, 220, 26, 276, 118, 242, 68, 773, 179, 508, 489, 794, 257, 286, 472, 424, 448, 823, 338, 151, 852, 679, 699, 642, 128, 819, 689, 485, 273, 306, 480, 431, 779, 32, 199, 187, 844, 90, 82, 372, 711, 134, 516, 309, 499, 86, 412, 722, 457, 527, 125, 792, 422, 850, 483, 777, 327, 238, 434, 462, 828, 438, 856, 230, 260, 104, 748, 569, 668, 329, 661, 75, 147, 604, 352, 60, 608, 871, 318, 716, 115, 469, 782, 221, 785, 835, 763, 9, 405, 224, 27, 267, 277, 735, 210, 119, 243, 444, 788, 69, 35, 289, 664, 676, 477, 774, 180, 744, 202, 509, 314, 21, 298, 490, 768, 543, 169, 50, 795, 240, 255, 284, 697, 640, 514, 420, 436, 258, 58, 733, 287, 475, 695, 473, 425, 627, 464, 830, 440, 599, 580, 449, 376, 409, 524, 858, 232, 366, 427, 158, 824, 339, 575, 740, 262, 152, 853, 47, 43, 680, 700, 629, 357, 106, 362, 838, 535, 643, 129, 820, 335, 566, 466, 832, 750, 78, 805, 190, 504, 690, 486, 274, 177, 307, 481, 432, 228, 780, 

In [67]:
''' SUffix array to Suffix tree '''

class Node(object):
    """Node class to be used in the SuffixArrayToTree class."""
    def __init__(self, parent_num):
        """Initialize parameters."""
        self.parent = parent_num
        self.children = []

    def update_parent(self, new_parent_num):
        """Update the nodes parent."""
        self.parent = new_parent_num

    def add_child(self, node_num):
        """Add a child node to the node."""
        self.children.append(node_num)

    def remove_child(self, node_num):
        """Remove a child from the node."""
        self.children.remove(node_num)


class Edge(object):
    """Edge class to be used in the SuffixArrayToTree class."""
    def __init__(self, start_index, stop_index):
        """Initialize parameters."""
        self.start_index = start_index
        self.stop_index = stop_index


class SuffixArrayToTree(object):
    """Constucts a suffix tree for thefrom the given word, suffix array and lcp array."""
    def __init__(self, word, suffix_array, lcp_array):
        """Initialize parameters and build the tree with the given words."""

        # Initialize parameters.
        self.word = word + ['', '$'][word[-1] != '$']
        self.suffix_array = suffix_array
        self.lcp_array = lcp_array

        # Initialize nodes and edges.
        self.nodes = [Node(-1)]
        self.edges = {}

        # Add the words to the generalized suffix tree.
        self._add_word()

    def _add_word(self):
        """Adds a word to the generalized suffix tree."""

        # Add each suffix to the generalized suffix tree.
        for i, sa_elmt in enumerate(self.suffix_array):
            # Get the insertion point and associated suffix.
            insertion_parent, insertion_suffix = self._insert_node(self.word[sa_elmt:], self.lcp_array[i])

            # Create the new node, and add it as a child to its parent node.
            self.nodes.append(Node(insertion_parent))
            self.nodes[insertion_parent].add_child(len(self.nodes)-1)

            # Create the edge associated to with the new node.
            self.edges[insertion_parent, len(self.nodes)-1] = Edge(len(self.word)-len(insertion_suffix), len(self.word))

    def _insert_node(self, suffix, lcp_distance, current_node=0):
        """Traverses the tree to determine the insertion point of the given suffix."""

        # Done if we've hit zero lcp_distance.
        if lcp_distance == 0:
            return current_node, suffix

        # Distance to the next node is the length of the edge word associated with travelling to rightmost path.
        dist_to_next_node = len(self.edge_word(self.edges[current_node, self.nodes[current_node].children[-1]]))

        if dist_to_next_node <= lcp_distance:
            return self._insert_node(suffix[dist_to_next_node:], lcp_distance - dist_to_next_node, self.nodes[current_node].children[-1])
        else:
            return self._split_edge(current_node, self.nodes[current_node].children[-1], lcp_distance), suffix[lcp_distance:]

    def _split_edge(self, parent_num, child_num, split_pos):
        """
        Splits the edge between the given parent and child nodes at the given split position.
        Inserts a new node at the split position and returns the index of the new node.
        """

        # Create the new node.
        new_node = len(self.nodes)
        self.nodes.append(Node(parent_num))
        self.nodes[new_node].add_child(child_num)

        # Add new_node as a child of parent_num.  Remove child_num from children list.
        self.nodes[parent_num].add_child(new_node)
        self.nodes[parent_num].remove_child(child_num)

        # Update child_num's parent to new_node.
        self.nodes[child_num].update_parent(new_node)

        # Create the new edges.
        old_edge = self.edges[parent_num, child_num]
        self.edges[parent_num, new_node] = Edge(old_edge.start_index, old_edge.start_index + split_pos)
        self.edges[new_node, child_num] = Edge(old_edge.start_index + split_pos, old_edge.stop_index)

        # Remove the old edge.
        del self.edges[parent_num, child_num]

        return new_node

    def edge_word(self, e):
        """Returns the substring associated with a given edge."""
        return self.word[e.start_index:e.stop_index]

    def word_up_to_node(self, node_num):
        """Returns the substring associated with a traversal to the given node."""
        node_word = ''
        while self.nodes[node_num].parent != -1:
            # Prepend the substring associated with each edge until we hit the root of the generalized suffix tree.
            node_word = self.edge_word(self.edges[self.nodes[node_num].parent, node_num]) + node_word
            node_num = self.nodes[node_num].parent

        return node_word

    def node_depth(self, node_num):
        """
        Returns the length of the substring traversal up to the given node,
        discounting the out of alphabet character, and without constructing the entire word.
        """

        # Trivially have depth zero if at the root.
        if node_num == 0:
            return 0

        # The first edge (working backwards) is the only one that can have an out of alphabet character, so take extra precaution.
        first_edge = self.edge_word(self.edges[self.nodes[node_num].parent, node_num])
        depth = len(first_edge) if '$' not in first_edge else len(first_edge[:first_edge.index('$')])

        # Move to the parent node and add the length of the next edge until we hit the root.
        node_num = self.nodes[node_num].parent
        while self.nodes[node_num].parent != -1:
            # Prepend the substring associated with each edge until we hit the root of the generalized suffix tree.
            depth += len(self.edge_word(self.edges[self.nodes[node_num].parent, node_num]))
            node_num = self.nodes[node_num].parent

        return depth

    
    
    
    
#LCP(Text), which stores the length of the longest common prefix shared by consecutive lexicographically ordered suffixes of Text
#lexicographically ordered suffixes of Text=['$', 'AGT$', 'GT$', 'GTAGT$', 'T$', 'TAGT$']
#LCP=[0, 0, 0, 2, 0, 1] 2: means two first letters shared by prefix at position 2 (GT$) and 3(GTAGT$)
seq='GTAGT$'
arr=[5, 2, 3, 0, 4, 1]
LCP=[0, 0, 0, 2, 0, 1]



#f=open('../../Downloads/dataset_296_7.txt','r')
f=open('../../Downloads/rosalind_ba9r.txt','r')
#f=open('input2.txt','r')

lines=f.read().splitlines()
f.close()
seq=lines[0]
arr=map(int,lines[1].split(', '))
LCP=map(int,lines[2].split(', '))
#print arr

Tree=SuffixArrayToTree(seq, arr, LCP)

fa=open('output.txt','w')
entry='\n'.join([Tree.edge_word(e) for e in Tree.edges.values()])
fa.write(entry)
fa.close()

In [144]:
''' Construct the Burrows-Wheeler transform of a string '''

def BWtransform(Text):
    Text=list(Text)
    Patterns=[]
    while True:
        first=Text.pop(-1)
        Text=[first]+Text
        #print Text
        Patterns+=[Text[:]]
        if Text[-1]=='$':
            break
    return ''.join([s[-1] for s in sorted(Patterns)])


def bwt(s):
    """Apply Burrows-Wheeler transform to input string."""
    table = sorted(s[i:] + s[:i] for i in range(len(s))) # Table of rotations of string
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string


#f=open('../../Downloads/dataset_297_5.txt','r')
f=open('../../Downloads/rosalind_ba9i.txt','r')
#f=open('input2.txt','r')
lines=f.read().splitlines()
f.close()

Text=lines[0]

Text='panamabananas$'

#Text='GCGTGCCTGGTCA$'

#print BWtransform(Text)
print bwt(Text)

smnpbnnaaaaa$a


In [240]:
''' Reconstruct a string from its Burrows-Wheeler transform'''



def StringFromBWT(bwt):
    bwt=list(bwt)
    bwtn=[bwt[x]+str(bwt[:x].count(bwt[x])+1) for x in range(len(bwt))] 
    fco=sorted(bwt)
    fcon=[fco[x]+str(fco[:x].count(fco[x])+1) for x in range(len(fco))] 
    #print fcon
    #print bwtn
    string=''
    charn=fcon[0]
    string+=charn[0]
    while True:
        charn=bwtn[fcon.index(charn)]
        #print charn
        if charn[0]=='$':
            break
        string+=charn[0]

    return string[::-1]


bwt='TTCCTAACG$A'
bwt="enwvpeoseu$llt"
'''
#f=open('../../Downloads/dataset_299_10.txt','r')
f=open('../../Downloads/rosalind_ba9j.txt','r')
#f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
bwt=lines[0]
'''
print StringFromBWT(bwt)

twelveplusone$


In [300]:
'''Implement BWMatching.'''

def BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst):
    '''
    Counts the total number of matches of Pattern in Text and output the start indices of Pattern in Text
    INPUT:
    FirstColumn=['$', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'm', 'n', 'n', 'n', 'p', 's']
    LastColumn=smnpbnnaaaaa$a
    Pattern=['a', 'n', 'a']
    LastToFirst=[13, 8, 9, 12, 7, 10, 11, 1, 2, 3, 4, 5, 0, 6]
    OUTPUT:
    #of match=3
    starting indices of match=[3, 4, 5]
    
    '''
    top = 0
    bottom = len(LastColumn)
    while top <= bottom:
        if len(Pattern)>0:
            symbol = Pattern.pop(-1) 
            if symbol in LastColumn[top:bottom+1]:
                topIndex = LastColumn[top:bottom+1].index(symbol)+top
                bottomIndex = top+len(LastColumn[top:bottom+1]) - 1 - LastColumn[top:bottom+1][::-1].index(symbol)
                top = LastToFirst[topIndex]
                bottom = LastToFirst[bottomIndex]

            else:
                return 0,[]
        else:
            return bottom-top +1, range(top,bottom+1)
            
            
                

#f=open('../../Downloads/dataset_300_8.txt','r')
f=open('../../Downloads/dataset_300_8.txt','r')

#f=open('../../Downloads/rosalind_ba9l.txt','r')
#f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
LastColumn=list(lines[0])
Patterns=[list(pat) for pat in lines[1].split(' ')]



#LastColumn='smnpbnnaaaaa$a'
#LastColumn=list('GCGTGCCTGGTCA$')
#Patterns=[list('ana')]

FirstColumn=sorted(LastColumn)
LC=[LastColumn[x]+str(LastColumn[:x].count(LastColumn[x])+1) for x in range(len(LastColumn))] 
FC=[FirstColumn[x]+str(FirstColumn[:x].count(FirstColumn[x])+1) for x in range(len(FirstColumn))]
LastToFirst=[FC.index(LC[i]) for i in range(len(LC))]




#print BWMatching(FirstColumn, LastColumn,list('ana') , LastToFirst)
result=[]
posi=[]
for Pattern in Patterns:
    print FirstColumn
    print LastColumn
    print LastToFirst
    print Pattern
    print BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst)
    result+=[BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst)[0]]
    posi+=BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst)[1]
print ' '.join(map(str,result))
#print ' '.join(map(str,posi))

['$', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'm', 'n', 'n', 'n', 'p', 's']
smnpbnnaaaaa$a
[13, 8, 9, 12, 7, 10, 11, 1, 2, 3, 4, 5, 0, 6]
['a', 'n', 'a']
(3, [3, 4, 5])
15


In [5]:
'''Generate the Last-to-First Mapping of a String'''


f=open('../../Downloads/rosalind_ba9k.txt','r')
lines=f.read().splitlines()
f.close()
LastColumn=list(lines[0])
n=int(lines[1])
#LastColumn='smnpbnnaaaaa$a'
print ''.join(LastColumn),n
#LastColumn=list('T$GACCA')
#n=3


FirstColumn=sorted(LastColumn)
LC=[LastColumn[x]+str(LastColumn[:x].count(LastColumn[x])+1) for x in range(len(LastColumn))] 
FC=[FirstColumn[x]+str(FirstColumn[:x].count(FirstColumn[x])+1) for x in range(len(FirstColumn))]
LastToFirst=[FC.index(LC[i]) for i in range(len(LC))]

print LastToFirst[n]

GCTGACGGGCTCGTGCGGAACGCTCTCATTTCTCCATGGCGGCTGATTGTAGGTGTCGGGTTGGACGCTCCTATTGGTTTAGATAGCCGTGGAAAGGGGGCGCAAATACAGGGAAAGAAAGGGCCTGCGCTATAGTCTTATAGTAAGCTCGAGTCCTCCATGAGGGGGTTACGCGGATAGGATACACGACCCAAAGCCTACTTGGCACGAGAGATACCCGGCGATACCATAGCCACGTCGCGCCATTTTTACCGACATGAAGCCCATGGAGAATTTATAAAGGTAGAACACCTCACCCCCACATAACTACGGTAGCACGCGCCCTTGACGAGCCGTTGGCCTGGAGGCCACTACCTAATTACCCCTTACGGTCGTTTTACGAGTGGGGATGGAGGGGCAGTGTAATTACAGGTCGGCAGGAATCCTCAGAGTATAGATGTCCCCTATCACGATTAGACCGATTAGAGTTTTGCGCGAGTGCTTCCACTTTTGCATATAAGTATGGCGGCAGACTTCCTCGACGTCGAACGTTTCAAGCAAGTATCAACGTACCGGAATGGTTGATAGAGGAATATCGTGCAAACAAAGGGATGGTCAGGACGCCCTATTGGGCTACGAGTAAACCCGCATATCCTTTGATGCCTGGTGCCACCGGGGCTACTCAAGTTTTCTAGAAAACTTGTTCACATATACCTACTTGCTTTTGATTTGGCAACTGTATTTATGGGTCCCACAAAATTCCCTAATACGTAAGGAGCGGCCCGCAAACATAGTCGGATTACCGATCGTGACCACCACAAGTCGGCTTCCGGCCTCTCAGGTGAGAAT$ 79
656


In [294]:
'''Pattern Matching with the Suffix Array'''
def PatternMatchingwithSA(Text,Patterns): 
    
    ''' 
    INPUT:
    Text=AATCGGGTTCAATCGGGGT$
    Patterns=['ATCG', 'GGGT']
    OUTPUT=[1, 4, 11, 15]
    '''
    
    def SuffixArray(text): 
        '''
        Construct the suffix array of a string.
        INPUT:
        seq=AACGATAGCGGTAGA$
        OUTPUT=[15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5] # suffixes in alphabetical order identified by the position on their first letter in string.
        '''
        d = {}
        for i in xrange(len(text)):
            d[i] = text[i:]
        return sorted(d.keys(), key=lambda x: d[x])

    def bwt(s):
        '''
        Apply Burrows-Wheeler transform to input string.
        INPUT:
        s=AATCGGGTTCAATCGGGGT$
        OUTPUT= TC$AATTTCGCGGGGGTAAG
        '''
        table = sorted(s[i:] + s[:i] for i in range(len(s)))
        last_column = [row[-1:] for row in table]
        return "".join(last_column)  

    def BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst):
        '''
        Counts the total number of matches of Pattern in Text and output the start indices of Pattern in Text
        INPUT:
        FirstColumn=['$', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'm', 'n', 'n', 'n', 'p', 's']
        LastColumn=smnpbnnaaaaa$a
        Pattern=['a', 'n', 'a']
        LastToFirst=[13, 8, 9, 12, 7, 10, 11, 1, 2, 3, 4, 5, 0, 6]
        OUTPUT:
        #of match=3
        starting indices of match=[3, 4, 5]
        '''
        top = 0
        bottom = len(LastColumn)
        while top <= bottom:
            if len(Pattern)>0:
                symbol = Pattern.pop(-1) 
                if symbol in LastColumn[top:bottom+1]:
                    topIndex = LastColumn[top:bottom+1].index(symbol)+top
                    bottomIndex = top+len(LastColumn[top:bottom+1]) - 1 - LastColumn[top:bottom+1][::-1].index(symbol)
                    top = LastToFirst[topIndex]
                    bottom = LastToFirst[bottomIndex]
                else:
                    return 0,[]
            else:
                return bottom-top +1, range(top,bottom+1)
    #########################################################
    LastColumn=list(bwt(Text))
    Patterns=[list(pat) for pat in Patterns]


    FirstColumn=sorted(LastColumn)
    LC=[LastColumn[x]+str(LastColumn[:x].count(LastColumn[x])+1) for x in range(len(LastColumn))] 
    FC=[FirstColumn[x]+str(FirstColumn[:x].count(FirstColumn[x])+1) for x in range(len(FirstColumn))]
    LastToFirst=[FC.index(LC[i]) for i in range(len(LC))]
    SA=SuffixArray(Text)

    posi=[]
    for Pattern in Patterns:
        posi+=BWMatching(FirstColumn, LastColumn, Pattern, LastToFirst)[1]

    return sorted([SA[n] for n in posi])
            
#f=open('../../Downloads/rosalind_ba9h (1).txt','r')
f=open('input.txt','r')

lines=f.read().splitlines()
f.close()
Text=lines[0]+'$'
Patterns=lines[1:]
print Text
print Patterns


print PatternMatchingwithSA(Text,Patterns)
print ' '.join(map(str,PatternMatchingwithSA(Text,Patterns)))

AATCGGGTTCAATCGGGGT$
['ATCG', 'GGGT']
[1, 4, 11, 15]
1 4 11 15
